In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler
from tensorflow.keras.optimizers import Adam, SGD
import keras_metrics
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import load_model 

import numpy as np
import pickle
import time

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/data_10slices/dataset with PGGAN/train",
    target_size=(256, 256),
    color_mode="grayscale",
    batch_size=8,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/data_10slices/dataset with PGGAN/train",
    target_size=(256, 256),
    color_mode="grayscale",
    batch_size=8,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="validation"
)

Found 15743 images belonging to 2 classes.
Found 3934 images belonging to 2 classes.


In [3]:
NAME = "dimentia-simple-CNN-{}".format(int(time.time()))
initializer = 'he_normal'
model = Sequential()

model.add(Conv2D(32, (5, 5), input_shape = (256, 256, 1), kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(512, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(1024, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1024, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(32, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(32, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1024, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary ()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 252, 252, 32)      832       
_________________________________________________________________
batch_normalization (BatchNo (None, 252, 252, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 32)      0         
_________________________________________________________________
activation (Activation)      (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 64)      18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 126, 126, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 64)        0

In [4]:
def lr_schedule(epoch):
    
    lr = 1e-4 #1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 50: # 120
        lr *= 1e-2
    elif epoch > 30: #80
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [5]:
tensorboard = TensorBoard(log_dir="/home/trojan/Desktop/dimentia/CNN_simple/logs/{}".format(NAME))

#es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=7)
#filepath="weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
#checkpoint = ModelCheckpoint('best_model_new_pggan.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()],
              )


Learning rate:  0.0001


In [6]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data = valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=tensorboard)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
   1/1967 [..............................] - ETA: 1s - loss: 0.9176 - accuracy: 0.5000 - precision: 0.5000 - recall: 0.5000 - auc: 0.3750WARNING:tensorflow:From /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
 375/1967 [====>.........................] - ETA: 49s - loss: 0.6808 - accuracy: 0.5805 - precision: 0.5733 - recall: 0.6155 - auc: 0.6214

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('best_model_new_pggan.h5')

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)